### Week 3 - Solutions
### Querying NCBI, Manipulating Sequences & Using BLAST @NCBI
- October 2023
- [https://https://github.com/tisimpson/bioinformatics1](https://github.com/tisimpson/bioinformatics1)
- [ian.simpson@ed.ac.uk](mailto:ian.simpson@ed.ac.uk)

### Setting up the Environment

In [ ]:
# Check that the essential libraries are installed
%pip install biopython
%pip install PrettyTable

# Import the Biopython functions we need.
# Remember that you can look up BioPython functions here - https://biopython.org/docs/latest/api/Bio.html
from Bio import SeqIO
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
from Bio import Entrez

# PrettyTable is a nice package for formatting tables - https://pypi.org/project/prettytable/
import prettytable
from prettytable import PrettyTable

#PLEASE DONT FORGET TO SET YOUR EMAIL THIS IS GOOD ETIQUETTE FOR A SERVICE USER
EMAIL = 'some.user@anywhere.com'

### Part 1A : Searching for Sequences at NCBI

In [ ]:
# In this section we're going to be using BioPython to query the NCBI nucleotide database

# Perform a search to obtain a list of gene ID's related to LHX2 Homo Sapiens gene
gene_name = 'LHX2'
species_name = 'Homo sapiens'

Entrez.email = EMAIL

# Here we use the contol keywords to target our search more accurately.
# These are very nicely detailed here so you should use this as a reference - https://www.ncbi.nlm.nih.gov/books/NBK49540/
search_string = gene_name+"[Gene] AND "+species_name+"[Organism] AND mRNA[Filter] AND RefSeq[Filter]"

# You can replicate this search on the NCBI website
# Go to https://www.ncbi.nlm.nih.gov/nucleotide/ and enter the search string:
# "LHX2"[Gene] AND "Homo sapiens"[Organism] AND mRNA[Filter] AND RefSeq[Filter]"

#Now we have a search string to seach for ids
handle = Entrez.esearch(db="nucleotide", term=search_string)

# This record variable holds the return result(s) from the search, note this can be a list of multiple entries
record = Entrez.read(handle)
ids = record['IdList']

# Check how many results were returned
print('Your search ('+search_string+') returned '+str(len(ids))+' hits')

# NB this returns the IDs of the sequences not the sequences themselves, so we need to actually pull the sequences using the eFetch function.

In [ ]:
#TASK ONE - Practice searching for these genes PAX6, SIX3, SHH.

# If you are going to be repeating the same task several times it can be helpful to define a function
def findHumanRNAs(gene_name):
    search_string = gene_name+"[Gene] AND \"Homo sapiens\"[Organism] AND mRNA[Filter] AND RefSeq[Filter]"
    handle = Entrez.esearch(db="nucleotide", term=search_string)
    record = Entrez.read(handle)
    ids = record['IdList']
    print('Your search ('+search_string+') returned '+str(len(ids))+' hits')

genes = ['PAX6','SIX3','SHH']

for gene in genes:
    findHumanRNAs(gene)

In [ ]:
#TASK TWO - Try searching for these same genes in Mouse [Mus musculus], Rat [Rattus norvegicus], and Chicken [Gallus galLus]
def findSpeciesRNAs(gene_name,species_name):
    search_string = gene_name+"[Gene] AND "+species_name+"[Organism] AND mRNA[Filter] AND RefSeq[Filter]"
    handle = Entrez.esearch(db="nucleotide", term=search_string)
    record = Entrez.read(handle)
    ids = record['IdList']
    return(ids,len(ids))

In [ ]:
# we can use a nested loop to do this efficiently
genes = ['PAX6','SIX3','SHH']
taxa = ['Mus musculus','Rattus norvegicus','Gallus gallus']

for taxon in taxa:
    print(f'These are the mRNA counts for the following',taxon,"genes")
    for gene in genes:
        ids,number = findSpeciesRNAs(gene,taxon)
        print(f"",gene,"has",number," mRNA transcripts")

### Part 1B : Retreiving Sequences from NCBI

In [ ]:
# Check the descriptions for each gene ID and load the DNA sequence of the closest match

Entrez.email = EMAIL

# For each ID returned above pull the database entry and print out the ID, the description, and any official accession IDs from the entry.
# This search specifies to look in the NCBI nucleotides database, with the IDs and to return the results in GenBank file format as plain text.
# The GenBank file format is very nicely explained here - https://www.ncbi.nlm.nih.gov/Sitemap/samplerecord.html
for gene_id in ids :
    handle = Entrez.efetch(db="nucleotide", id=gene_id, rettype="gb", retmode="text")
    gene = SeqIO.read(handle, "genbank")
    handle.close()
    print(gene_id,gene.description,gene.annotations['accessions'])

# There are lots of annotations to sequence records, but we're interested in accessions only here

In [ ]:
# TASK THREE - Experiment by just calling gene.annotations without the ['accessions'] selector and see what you get

def fetchRNAs(accessionID):
    handle = Entrez.efetch(db="nucleotide", id=accessionID, rettype="gb", retmode="text")
    mRNA = SeqIO.read(handle, "genbank")
    handle.close()
    return(accessionID,mRNA)

In [ ]:
# fetch only the first mRNA using its accession
accessionID,mRNA = fetchRNAs(ids[0])

# NB there are a lot of annotations these are held in a Python dict()
print(mRNA.annotations)

# you can print them all out
for index in mRNA.annotations.keys():
    print(f"",index,"-",mRNA.annotations[index])

In [ ]:
# TASK FOUR - Compare this to the sequence entry for these records on the NCBI website.

# e.g. for NM_204821 - https://www.ncbi.nlm.nih.gov/nuccore/NM_204821

### Part 1C : Selecting the Correct Sequences

In [ ]:
# Load the sequence we want for the experiment.
# If you remember we searched for RefSeq sequences only by using the RefSeq[Filter] term.
# Look at the following table - https://www.ncbi.nlm.nih.gov/books/NBK21091/table/ch18.T.refseq_accession_numbers_and_mole/?report=objectonly
# This shows that some of the RefSeq entries are high-quality still, but computationally predicted. You can see that what we really want here
# are sequences with accessions beginning "NM_" because these are the highest quality (normally) curated protein-coding sequences
# we can find these with a simple string search of our result annotations

import re

for gene_id in ids:
    handle = Entrez.efetch(db="nucleotide", id=gene_id, rettype="gb", retmode="text")
    gene = SeqIO.read(handle, "genbank")
    handle.close()
    # Here we use the re (regular expression) package to perform a search in the list looking for accession matches to "NM_"
    r = re.compile("NM_")
    hits = list(filter(r.match, gene.annotations['accessions']))
    if (hits):
        print(gene_id, gene.description, hits[0])

In [ ]:
# TASK FIVE - Here for simplicity I only take one hit to a RefSeq protein-coding transcript hit, but there could be several. Modfify the code 
# so that it could store multiple high quality (NM_) sequence IDs if they exist (HINT - make a new list)

import re

mRNAIDs,number = findSpeciesRNAs('SHH','Homo sapiens')

valid_mRNAs = []

for accessionID in mRNAIDs:
    accessionID,current_mRNA = fetchRNAs(accessionID)
    if(re.match("PREDICTED",current_mRNA.description)):
        print(f"mRNA sequence",''.join(current_mRNA.annotations['accessions']),"is computationally predicted")
    else:
        valid_mRNAs.append(''.join(mRNA.annotations['accessions']))
        print(f"mRNA sequence",''.join(current_mRNA.annotations['accessions']),"has been verified by NCBI staff")

print(f"Accession IDs for valid mRNAs are",','.join(valid_mRNAs))

### Part 2A : Exploring Sequence Objects

In [ ]:
# We're now going to take the hard-coded accession NM_004789 and fetch that explictly

handle = Entrez.efetch(db="nucleotide", id="NM_004789", rettype="gb", retmode="text")
gene = SeqIO.read(handle, "genbank")
handle.close()

# Display the sequence details
print(gene.id,gene.description,gene.annotations['accessions'])

# BioPython is able to access the features of the GenBank format file object

# # List all of the sequence feature (this is long) - commented out
# for f in gene.features:
#     print(f)

# Find the coding sequence and translate it
for f in gene.features:
    if f.type=='CDS':
        # show the details of the CDS elements
        print('Coding sequence at:',f.location)
        # convert the nucleotide coding sequence (remember this is an mRNA sequence record) into the protein sequence
        print(f.translate(gene).seq)

# This works by first iterating through all of the features in the BioSeq object and looking for ones that have type CDS
# In this case there is one CDS entry for the whole coding region
# We use the feature object function "translate" to convert the nucleotide sequence of the coding segment into a protein sequence.
# The way this works is that it operates on the whole BioSeq object and translated the region that corresponds to the feature only.
# The final .seq makes it return just the protein sequence as a new clean BioSeq object.

In [ ]:
# TASK SIX - Try extracting the protein sequences from tasks 1 and 2 above.

# let's just do one of the Six3 sequences

mRNAIDs,number = findSpeciesRNAs('SIX3','Homo sapiens')

accessionID,mRNA = fetchRNAs(mRNAIDs[0])

for feature in mRNA.features:
    if feature.type=='CDS':
        # show the details of the CDS elements
        print(f"Coding sequence is found at position",feature.location)
        # convert the nucleotide coding sequence (remember this is an mRNA sequence record) into the protein sequence
        protein_sequence = feature.translate(mRNA).seq
        print(f"The protein sequence is",len(protein_sequence),"amino acids long")
        print(f"with sequence",feature.translate(mRNA).seq)

### Part 2B : Working with Other Parts of Sequences

In [ ]:
# We can export other parts of the sequence too.

# Simply print the whole mRNA sequence
print ('The whole mRNA molecule is '+str(len(gene.seq))+' nucleotides long')
print(gene.seq)

# Find the coding sequence location again
for f in gene.features:
    if f.type=='CDS':
        # find the location of the coding sequence
        coding_location = f.location

# Because our sequence is an mRNA molecule anything that is downstream of the coding sequence is called the 3'-UTR (un-translated region)
# This region can have interesting and sometimes highly conserved sequences in that can do things like target an mRNA molecule to a particular 
# sub-cellular compartment.
# We can extract the 3-UTR by taking all the sequence after the coding sequence

seq3utr = gene.seq[coding_location.end:]

print('The 3-UTR is '+str(len(seq3utr))+' nucleotides long')
print(seq3utr)

In [ ]:
# TASK SEVEN - Try extracting the 5'-UTR (this is the part of the mRNA molecule that is upstream of the coding sequence), how long is it?

# this must start at the beginning of the sequence and end at the start of the coding sequence

seq5utr = gene.seq[:coding_location.start]

print('The 5-UTR is '+str(len(seq5utr))+' nucleotides long')
print(seq5utr)

In [ ]:
# TASK EIGHT - Try finding the lengths of the 5' and 3- UTRs of the sequences from tasks 1 and 2, above.

# let's do this for the SIX3 sequence we used above
mRNAIDs,number = findSpeciesRNAs('SIX3','Homo sapiens')

accessionID,mRNA = fetchRNAs(mRNAIDs[0])

for feature in mRNA.features:
    if feature.type=='CDS':
        # show the details of the CDS elements
        coding_location = feature.location
        seq3utr = mRNA.seq[coding_location.end:]
        print('The 3-UTR is '+str(len(seq3utr))+' nucleotides long')
        print(seq3utr)
        seq5utr = mRNA.seq[:coding_location.start]
        print('The 5-UTR is '+str(len(seq5utr))+' nucleotides long')
        print(seq5utr)

### Part 3A : Runing BLAST and Parsing Output

In [ ]:
# Now we're familiar with how to extract various bits from sequence objects that we download we can begin to use them as tools
# We're going to run a remote BLAST job at NCBI using BioPython

# To continue the example we are going to use the 3'-UTR sequence from LHX2 that we isolated above as the query sequence.

# We have many choices for the target database:
# nt - the full database
# refseq_rna - Curated (NM_, NR_) plus predicted (XM_, XR_) sequences from NCBI Reference Sequence Project
# refseq_genomic - Genomic sequences from NCBI Reference Sequence Project
# for available databases, see:
# ftp://ftp.ncbi.nlm.nih.gov/pub/factsheets/HowTo_BLASTGuide.pdf

# Set the database
database = 'refseq_rna'
# Specify our query, this is the simplest type - a blastn algorithm search of the refseq_rna database using the LHX2 3-UTR sequence
result_handle = NCBIWWW.qblast("blastn", database, seq3utr)
blast_record = NCBIXML.read(result_handle)

# NB These searches can take a while depending on how busy the servers are so you may need to wait for c.5 minutes.

In [ ]:
# NB the search above is simple, you can pass all the parameters to the search by modifying the qblast query string, for example:-

# > blast_handle = NCBIWWW.qblast("blastp", "nr",
#                               peptide_seq,
#                               expect=200000.0,
#                               filter=False,
#                               word_size=2,
#                               composition_based_statistics=False,
#                               matrix_name="PAM30",
#                               gapcosts="9 1",
#                               hitlist_size=1000)

# You can choose from pretty much all of the BLAST command line parameters which are listed here - https://www.ncbi.nlm.nih.gov/books/NBK279684/

In [ ]:
# Here we provide a small function that will take the results returned from your BLAST queries and formats it into a nice table

def result_handle_table(blast_record , query_len) :
    
    t = PrettyTable(['Description', 'Max Score' , 'Total Score' , 'Query Cover' , 'E Value' ,'Per Ident' , 'Acc Len' , 'Accession' ])
    t._max_width = {"Description" : 30}
    for alignment in blast_record.alignments: 
        score = 0
        max_score = 0
        query_cover = 0
        perc_ident = 100
        for hsp in alignment.hsps :
            score = score + hsp.bits
            max_score = max(max_score , hsp.bits)
            query_cover += ((hsp.query_end - hsp.query_start) / query_len)
            perc_ident = min(perc_ident , hsp.identities/hsp.align_length)
        if hsp.expect < 0.05 : 
            t.add_row([alignment.hit_def.split('>')[0] , round(score) , round(max_score) , '{:3.0f}%'.format(query_cover*100) , '{:1.2e}'.format(hsp.expect) , '{:3.2f}%'.format(perc_ident*100)  , alignment.length  , alignment.accession])
    
    return print(t[0:10])

In [ ]:
# You can use the function like this (below) but make sure you've already run the cell above first or it won't know the function.
result_handle_table(blast_record , len(seq3utr))

In [ ]:
# TASK NINE - Practice a variety of different search strategies with remote BLAST including adding additional parameters to control the search

# fetch the six3 protein sequence
mRNAIDs,number = findSpeciesRNAs('SIX3','Homo sapiens')

accessionID,mRNA = fetchRNAs(mRNAIDs[0])

for feature in mRNA.features:
    if feature.type=='CDS':
        # show the details of the CDS elements
        print(f"Coding sequence is found at position",feature.location)
        # convert the nucleotide coding sequence (remember this is an mRNA sequence record) into the protein sequence
        protein_sequence = feature.translate(mRNA).seq
        print(f"The protein sequence is",len(protein_sequence),"amino acids long")
        six3_protein = feature.translate(mRNA).seq

# Set the database
database = 'refseq_protein'

# perform a more specific BLAST search
blast_handle = NCBIWWW.qblast("blastp", "swissprot",
                              six3_protein,
                              expect=200000.0,
                              filter=False,
                              word_size=2,
                              composition_based_statistics=False,
                              matrix_name="PAM30",
                              gapcosts="9 1",
                              hitlist_size=1000)

blast_record = NCBIXML.read(blast_handle)

result_handle_table(blast_record , len(six3_protein))

In [ ]:
# TASK TEN - For at least some of the searches compare the tables produced from the function above to those from the search returned at the NCBI website.

# remember we can use prettytable to format the results of our BLAST search, you can re-use the code above to do that.

### Part 3B : Taking Advantage of NCBI Taxonomy

In [ ]:
# NCBI have a tremendous Taxonomy system that allows you to both specificlaly search based on species and to determine the species
# of sequences/results you have returned.

# Perform a search to obtain the taxid (taxonomic id - the equivalent of an accession number but for a species) for homo sapiens
organism = 'Homo sapiens' 
search_string = organism

#Now we have a search string to seach for ids
handle = Entrez.esearch(db="taxonomy", term=search_string)
record = Entrez.read(handle)
ids = record['IdList']
print(ids)

In [ ]:
# TASK ELEVEN - Try retrieving taxonomic ids for several other species, check them on the NCBI Taxonomy website - https://www.ncbi.nlm.nih.gov/taxonomy

modelOrganismSpecies = ['Escherichia coli', 'Saccharomyces cerevisiae', 'Drosophila melanogaster', 'Caenorhabditis elegans', 'Mus musculus', 'Homo sapiens']

# create an empty dictionary to store the taxon ids
taxon_ids = {}

# iterate through the list of species and retrieve the taxon id for each
for species in modelOrganismSpecies:
    # search for the species in the taxonomy database
    handle = Entrez.esearch(db="taxonomy", term=species)
    record = Entrez.read(handle)
    # store the taxon id in the dictionary
    taxon_ids[species] = record['IdList'][0]

# print the dictionary of taxon ids
print(taxon_ids)

### Part 4 : BLAST Challenges

Finally you can try to put together a strategy to address the questions below using the techniques you've been practicing above

* Perform a protein blast search to identify another organism with a fully function LHX2 gene
* Identify the function of the LHX2 gene
* Look for distantly related sequences and confirm in literature if this organism has a functioning LHX2 gene

Tips for challenges involve performing a range of different BLAST searches for example:
- searching different databases
- using different BLAST algorithms
- using different search parameters (such as different substitution matrices, remember the purposes of differnt PAM and BLOSUM matrices)
- try some of the more exotic BLAST searches at the NCBI webite such as PSI-BLAST and DELTA-BLAST

In [ ]:
# I'm just including an example of another type of BLAST search here, you can explore these final challenges yourselves basing off the code above
# and your experience using the BLAST suite at the NCBI website.

# We create a BLAST query in just the same way as above, but this time we are trying a more complicated strategy
# Here we try a tblastx strtegy starting with the 3'-UTR of LHX2 (non-coding remember)
# The system will perform a 6-frame translation of the nucleotide query and of the nucletoide RefSeq RNA target database
# It will try to find "protein" matches between the two.

# Database is set to RefSeq_RNA
database = 'refseq_rna'

# We next stitch together a query for a particular species in this case:
# "txid9606 [ORGN]' using the taxid we found for humans in the cell above.
# NB the slightly odd composition of this query term the organism specification is of the form "txidXXXX[ORGN]" where XXXX is the taxonommic ID number
# So for humans this would be 'txid9606[ORGN]'
organism = 'txid' + ids[0] + '[ORGN]'

# We now perform the BLAST search speficially limiting it to a given species. This would be very useful if you wanted to to species 
# specific BLAST searches.
result_handle = NCBIWWW.qblast("tblastx",
            database,
            seq3utr,
            gapcosts = '11 1',
            entrez_query = organism,
            expect=0.05,
            word_size=3,
            matrix_name="BLOSUM62",
            hitlist_size=100)

blast_record = NCBIXML.read(result_handle)

result_handle_table(blast_record , len(seq3utr))

# NB This is likely to be a very time-consuming search as it is a tblastx (36-way) query.